
# The files that you need to deal with are:
1.   VPTD_Dataset.csv  :: contains the ground truth for personality traits and sales  estimation
2.   smiles.csv :: contains the smiles detected in each video from the dataset.
3.   head_movements.json :: contains the info about the head movments (roll, pitch, yaw) for each frame




## Task 1: (2 marks)
Read the files and check their structure for further processing






In [ ]:
## Insert your code here
## You might use pandas and json to help you read the files

: 

## Task 2: (3 marks)
Show the correlation between smiles and Extraversion.


In [ ]:
## Insert your code here
## You might use a confusion matrix with correlation tests like (Pearson and Spearman)
## Try also finding any other relationship between smiles and (sales estimation and personality traits)

## Task 3: (5 marks)
Show the correlation between head movements and personality traits.


In [ ]:
## Insert your code here
## You should try first finding features like (mean, std, min, max, ...) and check any correlation
## You might use a confusion matrix.
## You can use the method shown in the lecture to find important events and then try to prove the correlation
## Try also finding any other relationship (use your way for analyzing)

## Task 4: (4 marks)
Prove the correlation you found between non-verbal cues (smiles and head movements) with personality traits, by training some models to predict the correlated traits depending on the extracted features.

In [ ]:
## Insert your code here
## You might train some models like (Random Forest, XGBoost, MLP, ...)
## Train you models on 70% of the data and test on the rest.
## Show your results and also compersion between the models.

#List of Papers:


1.   http://dx.doi.org/10.23919/FRUCT58615.2023.10143053
2.   http://dx.doi.org/10.3390/data8070113
3.   http://dx.doi.org/10.23919/FRUCT61870.2024.10516365

